# Heat PINN Validation — Demo Notebook

This notebook walks through both solvers and the validation pipeline.
It imports the project modules so there is zero code duplication.

In [ ]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent))

import numpy as np
import matplotlib.pyplot as plt
import torch
from IPython.display import Image

from heat_fd import HeatSolverFD
from heat_pinn import HeatPINN, train_pinn, evaluate_grid
from validate import l2_error, max_error, plot_error_heatmap, make_comparison_gif

## 1  Finite-Difference Solution

In [ ]:
fd = HeatSolverFD(Nx=100, Nt=5000, alpha=0.01)
print(f"CFL ratio r = {fd.r:.4f}")
u_fd = fd.solve()

fig, ax = plt.subplots(figsize=(7, 4))
im = ax.imshow(u_fd, aspect='auto', origin='lower', extent=[0,1,0,0.5], cmap='viridis')
fig.colorbar(im, ax=ax, label='u')
ax.set_xlabel('x'); ax.set_ylabel('t')
ax.set_title('FD solution')
plt.show()

## 2  PINN Training

In [ ]:
model = HeatPINN()
loss_history = train_pinn(model, epochs=5000, n_colloc=1000, verbose=True)

fig, ax = plt.subplots(figsize=(6, 4))
ax.semilogy(loss_history)
ax.set_xlabel('Epoch'); ax.set_ylabel('Loss')
ax.set_title('PINN training loss')
ax.grid(True, which='both', alpha=0.3)
plt.show()

## 3  Comparison

In [ ]:
u_pinn = evaluate_grid(model, fd.x, fd.t)

l2 = l2_error(u_fd, u_pinn)
mx = max_error(u_fd, u_pinn)
print(f"L2 error:  {l2:.6f}")
print(f"Max error: {mx:.6f}")

In [ ]:
plot_error_heatmap(u_fd, u_pinn, fname='../plots/error_heatmap.png')
Image(filename='../plots/error_heatmap.png')

In [ ]:
make_comparison_gif(u_fd, u_pinn, fd.t, fname='../plots/comparison.gif')
Image(filename='../plots/comparison.gif')